# Daten und Modelloptimierung - Teil 2

## Quelle der Daten

https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data (zuletzt aufgerufen: 01/2024)

https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29 (zuletzt aufgerufen: 01/2024)

## Installation der Bibliotheken

In [ ]:
# Installieren der keras_tuner-Bibliothek
%pip install keras_tuner

In [ ]:
# Bibliotheken importieren
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

import numpy as np

import matplotlib.pyplot as plt

import tensorflow as tf

import keras_tuner

from sklearn.model_selection import StratifiedKFold

## Einlesen der Daten

In [ ]:
# Daten laden und fehlende Werte entfernen
data_url = "https://github.com/timwgnd/Lehrbuch-Kuenstliche-Intelligenz-in-der-Medizin/raw/refs/heads/main/Brustkrebs.xlsx"
data = pd.read_excel(io=data_url, sheet_name = "Tabelle1")

# Entfernen von Zeilen mit fehlenden Werten
data = data.dropna()

# Anzeigen der ersten Zeilen des DataFrames, um einen Überblick über die Daten zu erhalten
print(data.head().to_markdown(index=False, tablefmt='psql'))

In [ ]:
# Diagnose-Werte in numerische Werte umwandeln
diagnosis_new = {"benign": 0, "malignant": 1}

data["diagnosis"] = data["diagnosis"].replace(diagnosis_new)

print(data.head().to_markdown(index=False, tablefmt='psql'))

## Aufteilung der Daten

In [ ]:
# Daten in Features (x) und Zielvariable (y) aufteilen
x = data.iloc[:, 1:]

y = data.iloc[:, 0]

In [ ]:
# Anzeigen der ersten Zeilen des Feature-Datensatzes (x)
print(x.head().to_markdown(index=False, tablefmt='psql'))

In [ ]:
# Anzeigen der ersten Zeilen der Zielvariablen (y)
print(y.head().to_markdown(index=False, tablefmt='psql'))

In [ ]:
# Anzeigen der ersten Zeilen der Zielvariablen (y)
print(y.value_counts().to_markdown(tablefmt='psql'))

## Principal Component Analysis

In [ ]:
# Skalieren der Feature-Daten (x) mit StandardScaler
scaler = StandardScaler()

scaler.fit(x)

scaled_data = scaler.transform(x)

In [ ]:
# Anwenden der Principal Component Analysis (PCA) mit
# 15 Komponenten auf die skalierten Daten
pca = PCA(n_components = 15)

pca.fit(scaled_data)

In [ ]:
# Erstellen eines Scree Plots zur Visualisierung der erklärten Varianz durch
# die Hauptkomponenten der PCA mit 15 Komponenten
PC_values = np.arange(pca.n_components_) + 1

plt.plot(PC_values, pca.explained_variance_ratio_, "o-", linewidth = 2)
plt.xticks(np.arange(1, len(PC_values)+1, 1))

plt.title("Scree Plot")
plt.xlabel("Principal Component")
plt.ylabel("Variance Explained")

In [ ]:
# Anwenden der Principal Component Analysis (PCA) mit 3 Komponenten auf die
# skalierten Daten, basierend auf dem Scree Plot
pca = PCA(n_components = 3)

pca.fit(scaled_data)

In [ ]:
# Erstellen eines Scree Plots für die PCA mit 3 Komponenten zur Bestätigung der Varianz
PC_values = np.arange(pca.n_components_) + 1

plt.plot(PC_values, pca.explained_variance_ratio_, "o-", linewidth = 2)
plt.xticks(np.arange(1, len(PC_values)+1, 1))

plt.title("Scree Plot")
plt.xlabel("Principal Component")
plt.ylabel("Variance Explained")

In [ ]:
# Transformieren der skalierten Daten mit der PCA, reduziert auf 3 Hauptkomponenten
x_pca = pca.transform(scaled_data)

print(x_pca)

In [ ]:
# Visualisieren der ersten beiden Hauptkomponenten (PC1 und PC2) der
# PCA-transformierten Daten, eingefärbt nach der Diagnose
plt.figure(figsize = (8,6))

plt.scatter(x_pca[:,0],x_pca[:,1], c = data["diagnosis"])

plt.xlabel("PC1")
plt.ylabel("PC2")

In [ ]:
# Aufteilen der PCA-transformierten Daten und der Zielvariablen in Trainings- und Testsets
# für das Modelltraining
x_train, x_test, y_train, y_test = train_test_split(x_pca, y, test_size = 0.15)

## Erstellen, Trainieren und Evaluieren des KI-Modells

In [ ]:
# Erstellen eines ersten neuronalen Modells
model_1 = tf.keras.models.Sequential()

model_1.add(tf.keras.layers.Dense(64, activation = tf.nn.relu))
model_1.add(tf.keras.layers.Dense(256, activation = tf.nn.relu))
model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model_1.add(tf.keras.layers.Dense(2, activation = tf.nn.softmax))

In [ ]:
# Kompilieren des ersten Modells mit SGD-Optimizer und Sparse Categorical Crossentropy-Loss
model_1.compile(
    optimizer = "SGD",
    loss = "sparse_categorical_crossentropy",
    metrics = ["accuracy"]
)

In [ ]:
# Trainieren des ersten Modells mit den Trainingsdaten für 10 Epochen
model_1.fit(x_train, y_train, epochs = 10)

In [ ]:
# Evaluieren des ersten Modells mit den Testdaten
eval_results = model_1.evaluate(x_test, y_test)
print("[test loss, test accuracy]:", eval_results)

In [ ]:
# Erstellen des zweiten neuronalen Modells mit Regularisierung
model_2 = tf.keras.models.Sequential()

model_2.add(tf.keras.layers.Dense(64, activation = tf.nn.relu,
                                kernel_initializer = "he_uniform",
                                kernel_regularizer = tf.keras.regularizers.L1(0.01),
                                bias_regularizer = tf.keras.regularizers.L2(0.01)))
model_2.add(tf.keras.layers.Dense(256, activation = tf.nn.relu))
model_2.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model_2.add(tf.keras.layers.Dense(2, activation = tf.nn.softmax))

In [ ]:
# Kompilieren des zweiten Modells mit SGD-Optimizer und Sparse Categorical Crossentropy-Loss
model_2.compile(
    optimizer = "SGD",
    loss = "sparse_categorical_crossentropy",
    metrics = ["accuracy"]
)

In [ ]:
# Trainieren des zweiten Modells mit den Trainingsdaten für 5 Epochen
model_2.fit(x_train, y_train, epochs = 10)

In [ ]:
# Evaluieren des zweiten Modells mit den Testdaten
eval_results = model_2.evaluate(x_test, y_test)
print("[test loss, test accuracy]:", eval_results)

## Hyperparameter-Optimierung mit Random Search

In [ ]:
# Funktion zum Erstellen eines Keras-Modells für die Hyperparameter-Optimierung
def create_model(hp):
    model = tf.keras.models.Sequential()

    # Definiere zu testende Hyperparameter-Einstellungen
    hp_units = hp.Int("units", min_value = 64, max_value = 128, step = 32)
    hp_loss = hp.Choice("loss", ["sparse_categorical_crossentropy", "MSE"])
    hp_optimizer = hp.Choice("optimizer", ["adam", "SGD"])

    # Eingangsform basierend auf der Anzahl der PCA-Komponenten festlegen
    model.add(tf.keras.layers.Dense(units=hp_units, activation = tf.nn.relu, input_shape=(3,)))
    model.add(tf.keras.layers.Dense(256, activation = tf.nn.relu))
    model.add(tf.keras.layers.Dense(units=hp_units, activation = tf.nn.relu))

    model.add(tf.keras.layers.Dense(2, activation = tf.nn.softmax))

    model.compile(optimizer = hp_optimizer, loss = hp_loss, metrics = ["accuracy"])

    return model

In [ ]:
# Erstellen eines Random-Search Tuners
tuner = keras_tuner.RandomSearch(
    hypermodel=create_model,
    objective='val_accuracy',
    max_trials=5
)

In [ ]:
# Durchführen der Random Search zur Hyperparameter-Optimierung des Modells
tuner.search_space_summary()
tuner.search(x_train, y_train, epochs = 10, validation_split = 0.2)

In [ ]:
# Ermitteln des besten Hyperparameter-Settings
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(best_hps.values)

In [ ]:
# Erstellen eines finalen Modells mit den besten Hyperparametern
# und anschließendes Trainieren auf den Daten für 50 Epochen
model_tuned = tuner.hypermodel.build(best_hps)
history = model_tuned.fit(x_train, y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
# Plotten der Trainings- und Testgenauigkeit
plt.plot(model_tuned.history.history["accuracy"], label = "train_accuracy")
plt.plot(model_tuned.history.history["val_accuracy"], label = "test_accuracy")
plt.legend()
plt.show()

In [ ]:
# Evaluieren des finalen Modells mit den Testdaten
eval_results = model_tuned.evaluate(x_test, y_test)
print("[test loss, test accuracy]:", eval_results)